In [77]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json

In [78]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openai=OpenAI()
MODEL = 'gpt-4o-mini'

In [92]:
system_message="""
You are a friendly and professional chat assistant for **Tailor_Khay**, a fashion design brand. Your role is to handle customer inquiries, take orders, and provide excellent customer service.
You also help clients book appointments by checking available dates.You help clients book appointments by checking available dates.
Initial Customer Greeting Protocol
** start by asking if the customer is a returning Tailor_Khay customer:**

- **If they are a returning customer:** Reply with "You na my person, you too sure! 😊"
- **If they are NOT a returning customer:** Reply with "Heyy boo, you are at the right place! ✨"

## Services & Pricing Information

### Basic Style Price Ranges (Starting prices - not fixed):
- **Iro and Buba:** ₦4,000
- **Two Piece:** ₦20,000  
- **Boubou:** ₦7,500
- **Abaya:** ₦10,000

**Important:** Always inform customers that these are starting price ranges and final pricing may vary based on design complexity, fabric choice, and customizations.

### Additional Fees:
- **Rush Orders (less than 10 days):** Extra ₦5,000
- **Fabric Sourcing Service:** Extra ₦5,000 (if Tailor_Khay provides the fabric)
Standard Timeline:
Regular Orders:** 10 working days completion time
Order Taking Process

When taking an order, ask the following questions:

1. **Style Preference:**
   - "What style are you looking to sew?"
   - "Do you have a specific design in mind?"
   - Offer to show available styles if needed

2. **Color Preferences:**
   - "What color(s) would you prefer?"
   - Ask for primary and accent colors if applicable

3. **Fabric Arrangements:**
   - "Will you be providing your own fabric, or would you like us to source it for you?"
   - If they want fabric sourcing, inform about the ₦5,000 additional fee

4. **Timeline Requirements:**
   - "When do you need this completed?"
   - If less than 10 days, inform about the ₦5,000 rush fee

5. **Measurements & Fitting:**
   - Schedule measurement appointment
   - Confirm fitting arrangements
Special Protocol for Asoebi Orders
IMMEDIATE ACTION REQUIRED:
If a customer mentions **"asoebi"** or asks about group/matching outfits:
- Respond: "Please hold on, let me connect you with Tailor_Khay directly for your asoebi order. This requires special attention! 👗"
- Transfer the chat to the main designer immediately
Complaint Handling
- Listen actively and empathetically
- Apologize for any inconvenience
- Gather all relevant details
- Provide solutions or escalate to Tailor_Khay when necessary
- Follow up to ensure customer satisfaction
Tone & Communication Style
- Be warm, friendly, and professional
- Use Nigerian expressions appropriately
- Show enthusiasm for fashion and design
- Be clear about pricing and timelines
- Always confirm order details before finalizing
Key Reminders
- Always confirm if returning customer first
- Emphasize that basic prices are starting ranges
- Mention 10-day standard timeline
- Clarify additional fees upfront
- Transfer asoebi orders immediately to main designer
- Be helpful and solution-oriented for complaints
Remember: You represent the Tailor_Khay brand - be professional, creative, and customer-focused in all interactions!

"""

In [93]:
def chat_tailorkhay(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [94]:
gr.ChatInterface(fn=chat_tailorkhay, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


In [95]:
calendar = {
    "2025-08-05": "booked",
    "2025-08-06": "available",
    "2025-08-07": "available",
    "2025-08-08": "booked",
    "2025-08-09": "available",
    "2025-08-10": "booked",
    "2025-08-11": "available",
    "2025-08-12": "available",
    "2025-08-13": "booked",
    "2025-08-14": "available",
    "2025-08-15": "booked",
    "2025-08-16": "available",
    "2025-08-17": "available",
    "2025-08-18": "booked",
    "2025-08-19": "available"
}


In [96]:
def check_available_dates(start_date=None, end_date=None):
    available = {}
    for date, status in calendar.items():
        if status == "available":
            if (not start_date or date >= start_date) and (not end_date or date <= end_date):
                available[date] = status
    return list(available.keys())



In [97]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_available_dates",
            "description": "Check available appointment dates for Khay the tailor",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "The start of the date range to check (YYYY-MM-DD)"
                    },
                    "end_date": {
                        "type": "string",
                        "description": "The end of the date range to check (YYYY-MM-DD)"
                    }
                },
                "required": []
            }
        }
    }
]



In [98]:
#tools = [{"type": "function", "function": date_check_func}]


In [99]:
def chat_tailorkhay(message, history):
    messages = [{"role": "system", "content":system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_response, _ = handle_tool_call(message)

        messages.append(message)
        messages.append(tool_response)

        # Re-run with tool result
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )

    return response.choices[0].message.content




In [100]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    start = arguments.get('start_date')
    end = arguments.get('end_date')
    available_dates = check_available_dates(start, end)

    response = {
        "role": "tool",
        "content": json.dumps({"available_dates": available_dates}),
        "tool_call_id": tool_call.id
    }
    return response, (start, end)

In [102]:
chat_tailorkhay("I want to book an appointment between 2025-08-03 and 2025-08-017", [])


"I have found several available dates for appointments between 2025-08-03 and 2025-08-17. Please let me know which date works best for you:\n\n- 2025-08-06\n- 2025-08-07\n- 2025-08-09\n- 2025-08-11\n- 2025-08-12\n- 2025-08-14\n- 2025-08-16\n- 2025-08-17\n\nLet me know your preference and I'll book it for you! 😊"

In [103]:
gr.ChatInterface(fn=chat_tailorkhay, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
